In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 1. Загрузка и обработка данных

Загрузим данные поездок за май 2016 года и посмотрим на них:

In [2]:
%%time
May = pd.read_csv('Data/yellow_tripdata_2016-05.csv')
May.head()

CPU times: user 56.5 s, sys: 4.58 s, total: 1min 1s
Wall time: 1min 11s


In [3]:
May.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-05-01 00:00:00,2016-05-01 00:17:31,1,3.60,-73.985901,40.768040,1,N,-73.983986,40.730099,1,15.0,0.5,0.5,1.50,0.0,0.3,17.80
1,2,2016-05-01 00:00:00,2016-05-01 00:07:31,1,1.68,-73.991577,40.744751,1,N,-73.975700,40.765469,1,7.5,0.5,0.5,0.88,0.0,0.3,9.68
2,2,2016-05-01 00:00:00,2016-05-01 00:07:01,6,1.09,-73.993073,40.741573,1,N,-73.980995,40.744633,1,6.5,0.5,0.5,1.56,0.0,0.3,9.36
3,2,2016-05-01 00:00:00,2016-05-01 00:19:47,1,4.21,-73.991943,40.684601,1,N,-74.002258,40.733002,1,17.0,0.5,0.5,3.66,0.0,0.3,21.96
4,2,2016-05-01 00:00:00,2016-05-01 00:06:39,1,0.56,-74.005280,40.740192,1,N,-73.997498,40.737564,1,6.0,0.5,0.5,1.46,0.0,0.3,8.76


In [4]:
May.shape

(11836853, 19)

Произведем фильтрацию данных: удалим поездки с нулевым временем, нулевым расстоянием, нулевым количеством пассажиров и координатами начала за пределами Нью-Йорка

In [5]:
west = -74.25559
east = -73.70001
south = 40.49612
nord = 40.91553

In [10]:
bad_indices = ((May['tpep_pickup_datetime'] == May['tpep_dropoff_datetime']) | (May['passenger_count'] == 0) |\
             (May['trip_distance'] == 0) | (May['pickup_longitude'] < west) | (May['pickup_longitude'] > east) |\
               (May['pickup_latitude'] < south) | (May['pickup_latitude'] > nord)).nonzero()
print type(bad_indices), len(bad_indices), len(bad_indices[0])

<type 'tuple'> 1 210332


In [23]:
May.drop(bad_indices[0], inplace = True)
May.shape

(11626521, 19)

In [24]:
a = May['tpep_pickup_datetime'][0]
print type(a)

<type 'str'>


In [26]:
print a[:-3]

2016-05-01 00:00


In [ ]:
May['tpep_pickup_datetime'] = pd.Series([x[:-3] for x in May['tpep_pickup_datetime'].values])